# Memory and RAG

特定のステップ直前にエージェントのコンテキストにインテリジェントに追加できる有用な事実のストアを維持することが価値ある複数のユースケースが存在します。ここでの典型的なユースケースは、データベースから関連情報を取得し、その情報をエージェントのコンテキストに追加するRAGパターンです。

AgentChat は、この機能を提供するために拡張可能な Memory プロトコルを提供しています。主要なメソッドは、query、update_context、add、clear、および close です。  

- add： メモリストアに新しいエントリを追加します  
- query： メモリストアから関連する情報を取得します  
- update_context： 取得した情報をエージェントの内部モデルコンテキストに追加して変更します（AssistantAgentクラスで使用されます）  
- clear： メモリストアからすべてのエントリを削除します  
- close： メモリストアが使用しているリソースを解放します

In [ ]:
import os  
from dotenv import load_dotenv  
  
from autogen_agentchat.agents import AssistantAgent  
from autogen_agentchat.teams import RoundRobinGroupChat  
from autogen_agentchat.conditions import TextMessageTermination  
from autogen_core import CancellationToken  
from autogen_ext.models.openai import AzureOpenAIChatCompletionClient  
from autogen_agentchat.ui import Console
from autogen_ext.tools.mcp import SseServerParams, mcp_server_tools  

from autogen_core.memory import ListMemory, MemoryContent, MemoryMimeType

load_dotenv()

## OpenTelemetry によるトレーサーのセット
マルチエージェントのデバッグには OpenTelemetry によるトレーサーを利用すると便利。`OpenAIInstrumentor` を使用して OpenAI コールをキャプチャできます。ここではトレース UI として [Jaeger](https://www.jaegertracing.io/download/) を使用しています。

In [ ]:
from opentelemetry import trace
from opentelemetry.sdk.resources import Resource
from opentelemetry.sdk.trace import TracerProvider
from opentelemetry.sdk.trace.export import BatchSpanProcessor
from opentelemetry.exporter.otlp.proto.grpc.trace_exporter import OTLPSpanExporter
from opentelemetry.instrumentation.openai import OpenAIInstrumentor

service_name = "autogen"

# OTLPエクスポーターの設定 (gRPC経由で送信)
otlp_exporter = OTLPSpanExporter(
    endpoint="http://localhost:4317",  # JaegerのgRPCエンドポイント
)
tracer_provider = TracerProvider(resource=Resource({"service.name": service_name}))
    
# トレーサープロバイダーの設定
trace.set_tracer_provider(tracer_provider)

# バッチスパンプロセッサーを設定
span_processor = BatchSpanProcessor(otlp_exporter)
tracer_provider.add_span_processor(span_processor)

# トレーサーを取得
tracer = tracer_provider.get_tracer(service_name)

OpenAIInstrumentor().instrument()

In [ ]:
mcp_server_uri = os.getenv("MCP_SERVER_URI")
azure_openai_key = os.getenv("AZURE_OPENAI_KEY")
azure_openai_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
azure_openai_model = os.getenv("AZURE_OPENAI_MODEL")
azure_deployment = os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT")
api_version = os.getenv("AZURE_OPENAI_API_VERSION")
openai_model_name = os.getenv("OPENAI_MODEL_NAME")

## Tools の定義(MCP over SSE)

In [ ]:

# Set up the OpenAI/Azure model client  
model_client = AzureOpenAIChatCompletionClient(  
    api_key=azure_openai_key,  
    azure_endpoint=azure_openai_endpoint,  
    api_version=api_version,  
    azure_deployment=azure_deployment,  
    model=openai_model_name,  
)  

## Memory 実装
記憶を時系列順に保持し、最も最近の記憶をモデルのコンテキストに追加するシンプルなリストベースの記憶実装です。この実装は、シンプルで予測可能な設計となっており、理解やデバッグが容易です。以下の例では、ListMemoryを使用してユーザーの設定を保持するメモリバンクを実装し、エージェントの応答に一貫したコンテキストを提供する方法示します。



In [ ]:

# Initialize user memory
user_memory = ListMemory()

# Add user preferences to memory
await user_memory.add(MemoryContent(content="喋る時は関西弁を好む", mime_type=MemoryMimeType.TEXT))

await user_memory.add(MemoryContent(content="批判的な観点から指摘されるのを嫌う", mime_type=MemoryMimeType.TEXT))

await user_memory.add(MemoryContent(content="出力形式は箇条書きで3つにまとめないと満足しない", mime_type=MemoryMimeType.TEXT))

user_memory.content

## エージェント定義

In [ ]:
# Set up the assistant agent  
agent = AssistantAgent(  
    name="ai_assistant",  
    model_client=model_client,  
    system_message=(  
        "あなたは役立つアシスタントです。複数のツールを使用して情報を検索し、質問に回答することができます。"  
        "利用可能なツールを確認し、必要に応じて使用してください。ユーザーが不明点がある場合は、確認のための質問をすることもできます。"
    ),
    memory=[user_memory]
)  


In [ ]:
# Run the agent with a task.
stream = agent.run_stream(task="今後のエンタープライズAIエージェント市場について君はどう考える？",)
await Console(stream)


# Task-Centric Memory
AutoGen で実験的・研究的機能として実装中の Task-Centric Memory は、AI エージェントが「ユーザーから与えられた助言・ルール・ヒント・デモ・失敗から得た知見」などを“記憶”し、将来の類似タスクや会話でそれらを自動的に活用できるようにするためのメモリー機構です。

タスクとは、アプリがエージェントに与えるテキスト指示のことです。インサイトとは、エージェントがそのようなタスクを実行するのに役立つ可能性のあるテキスト（ヒント、アドバイス、デモンストレーション、計画など）のことです。

## Task-Centric Memory の特徴
- 「タスク解決・学習」に特化したメモリ設計
- ユーザーのアドバイス・デモ・エージェント自身の失敗から得た インサイト（知見） を「タスク」と紐付けて保存
- 類似タスクへの応用・一般化・自己学習を重視
- MemoryController や Teachability など、学習ループや知見抽出・再利用のための高度なロジックを持つ


In [ ]:
!pip install "autogen-ext[task-centric-memory]"

## 1. 直接的な記憶の保存と検索

In [ ]:
from autogen_ext.models.openai import OpenAIChatCompletionClient,AzureOpenAIChatCompletionClient
from autogen_ext.experimental.task_centric_memory import MemoryController

memory_controller = MemoryController(reset=True, client=model_client)

# 1. タスクとインサイト（ヒント/教え）を登録
await memory_controller.add_memo(task="友人Aさんに子供が生まれたので出産祝いを贈りたい", insight="A様のご友人は2人目のお子様とのことで、既に基本的な育児グッズは揃っている可能性がある")

# 2. 新しいタスクに対して関連インサイトを検索
new_task = "友人Aに出産祝いを贈りたいんですが、何がいいですか？"
memos = await memory_controller.retrieve_relevant_memos(task=new_task)

print(f"Task: {new_task}")
print("Retrieved insights:")
for memo in memos:
    print("-", memo.insight)


### 記憶の永続化
`MemoryController(reset=False)` とすることで `MemoryBank(pickle, ChromaDB)` に永続化されます。入力（タスクやトピック）に対して、類似するメモ（インサイト）をベクトル検索で高速に取得できるようになります。デフォルト実装では抽出したタスクを ChromaDB に保存しベクトル検索、タスクに対応するインサイトを pickle からロードするという実装になっています。

タスク自体をベクトル化し、類似検索することで、過去の“教え”や“インサイト”を“広範な未経験タスク”にも応用できるという仕組みになっています。

## 2. ユーザーのアドバイスや修正から学ぶ

では、このタスクとインサイトのセットを会話から自動的に抽出するにはどのように実装すれば良いのでしょうか。

AssistantAgent に Teachability クラスをセットすることでユーザーが入力した発言の中から「教え（助言・ルール）」が自動で抽出・保存されるようになっています。

例えば、以下のような発言をしてみましょう。
- `要約タスクは必ず3行でお願いします`
- `要約タスクは関西弁でしゃべる必要があります`
- `あなたは何を記憶していますか？`

In [ ]:
from autogen_ext.experimental.task_centric_memory.utils import Teachability

# Create an instance of Task-Centric Memory, passing minimal parameters for this simple example
memory_controller = MemoryController(reset=False, client=model_client)

# Wrap the memory controller in a Teachability instance
teachability = Teachability(memory_controller=memory_controller)

memory_controller.reset_memory()

# Create an AssistantAgent, and attach teachability as its memory
assistant_agent = AssistantAgent(
    name="teachable_agent",
    system_message = "あなたは過去の会話からユーザーの指示を記憶する特別な能力を持つ、役立つAIアシスタントです。",
    model_client=model_client,
    memory=[teachability],
)

# Enter a loop to chat with the teachable agent
print("現在、学習可能なエージェントとチャット中です。最初のメッセージを入力してください。終了するには「exit」または「quit」と入力してください。")
while True:
    user_input = input("\nYou: ")
    if user_input.lower() in ["exit", "quit"]:
        break
    await Console(assistant_agent.run_stream(task=user_input))

# Close the connection to the client
await model_client.close()


In [ ]:
import pickle

# ファイルをバイナリ読み込みモードで開く
with open("./memory_bank/default/string_map/uid_text_dict.pkl", "rb") as f:
    data = pickle.load(f)

data

In [ ]:
# ファイルをバイナリ読み込みモードで開く
with open("./memory_bank/default/uid_memo_dict.pkl", "rb") as f:
    data = pickle.load(f)

data

In [ ]:
import sqlite3

# 1. データベースファイルに接続
conn = sqlite3.connect("./memory_bank/default/string_map/chroma.sqlite3")

# 2. カーソルを取得
cur = conn.cursor()

# 3. 特定テーブルの中身をすべて表示
cur.execute("SELECT seq_id, metadata, encoding FROM embeddings_queue;")  # ← mytable を参照したいテーブル名に変更
rows = cur.fetchall()

# 行データを表示
for row in rows:
    print(row)

# 6. 終了処理
cur.close()
conn.close()